# BLACK_SCHOLES

## Overview
The `BLACK_SCHOLES` function calculates the theoretical price of a European call or put option using the Black-Scholes formula. This enables users to quickly and accurately price options for financial analysis, risk management, or portfolio valuation.

The Black-Scholes model is a mathematical model for pricing European-style options and is widely used in financial engineering. The model assumes that the price of the underlying asset follows a geometric Brownian motion with constant drift and volatility. The key calculations performed by the `black_scholes` function are:

- **Call Option Price:** The price to buy the right, but not the obligation, to purchase an asset at a specified strike price (K) on the option's expiration date. The Black-Scholes formula for a European call option is:

```math
C = S N(d_1) - K e^{-rT} N(d_2)
```

- **Put Option Price:** The price to buy the right, but not the obligation, to sell an asset at a specified strike price (K) on the option's expiration date. The Black-Scholes formula for a European put option is:

```math
P = K e^{-rT} N(-d_2) - S N(-d_1)
```

where:
- $S$: Current price of the underlying asset (e.g., stock price)
  - $K$: Strike price of the option
  - $T$: Time to expiration in years
  - $r$: Annual risk-free interest rate (as a decimal)
  - $\sigma$: Volatility of the underlying asset (annualized standard deviation, as a decimal)
  - $N(\cdot)$: Cumulative distribution function of the standard normal distribution
  - $d_1 = \frac{\ln(S/K) + (r + 0.5 \sigma^2) T}{\sigma \sqrt{T}}$
  - $d_2 = d_1 - \sigma \sqrt{T}$

- **Assumptions:**
  - The option is European (can only be exercised at expiration)
  - No dividends are paid out during the life of the option
  - Markets are efficient (market movements cannot be predicted)
  - No commissions or transaction costs
  - The risk-free rate and volatility of the underlying asset are constant
  - The returns of the underlying asset are normally distributed

## Usage
To use the `BLACK_SCHOLES` function in Excel, enter it as a formula in a cell, specifying the required parameters:

```excel
=BLACK_SCHOLES(S, K, T, r, sigma, option_type)
```

## Arguments
| Argument      | Type    | Required | Description                                                        | Example   |
|--------------|---------|----------|--------------------------------------------------------------------|-----------|
| S            | float   | Yes      | Current price of the underlying asset (e.g., stock price)          | 100       |
| K            | float   | Yes      | Strike price of the option                                         | 105       |
| T            | float   | Yes      | Time to expiration in years (e.g., 0.5 for 6 months)               | 0.5       |
| r            | float   | Yes      | Annual risk-free interest rate (as a decimal, e.g., 0.05 for 5%)   | 0.03      |
| sigma        | float   | Yes      | Volatility of the underlying asset (annualized std dev, decimal)   | 0.2       |
| option_type  | string  | Yes      | Type of option: 'call' for Call option, 'put' for Put option       | "call"   |

## Returns
| Return Value | Type  | Description                                      | Example                 |
|--------------|-------|--------------------------------------------------|-------------------------|
| price        | float | Theoretical price of the option (call or put)     | 4.18                    |
| error        | string| Error message if calculation fails                | "Error: Invalid input"  |

## Examples
### Pricing a European Call Option
A financial analyst wants to price a 6-month European call option on a stock currently trading at $100, with a strike price of $105, a risk-free rate of 3%, and an annual volatility of 20%.

```excel
=BLACK_SCHOLES(100, 105, 0.5, 0.03, 0.2, "call")
```

### Pricing a European Put Option
A portfolio manager wants to evaluate the cost of buying a 1-year European put option to hedge against a potential decline in a stock currently priced at $50, with a strike price of $45, a risk-free rate of 2%, and a volatility of 25%.

```excel
=BLACK_SCHOLES(50, 45, 1, 0.02, 0.25, "put")
```


In [ ]:
import math
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type):
    """
    Calculate the Black-Scholes price for a European call or put option.

    Args:
        S (float): Current price of the underlying asset
        K (float): Strike price
        T (float): Time to expiration in years
        r (float): Annual risk-free interest rate (decimal)
        sigma (float): Volatility of the underlying asset (decimal)
        option_type (str): 'call' or 'put'

    Returns:
        float: Theoretical price of the option
        or
        str: Error message if calculation fails
    """
    if not (isinstance(S, (int, float)) and S > 0):
        return "Error: S must be a positive float"
    if not (isinstance(K, (int, float)) and K > 0):
        return "Error: K must be a positive float"
    if not (isinstance(T, (int, float)) and T > 0):
        return "Error: T must be a positive float"
    if not (isinstance(r, (int, float)) and r >= 0):
        return "Error: r must be a non-negative float"
    if not (isinstance(sigma, (int, float)) and sigma > 0):
        return "Error: sigma must be a positive float"
    if option_type not in ('call', 'put'):
        return "Error: option_type must be 'call' or 'put'"

    try:
        d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
        d2 = d1 - sigma * math.sqrt(T)
        if option_type == 'call':
            price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
        else:  # put
            price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return float(price)
    except Exception:
        return "Error: Invalid input"

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_call_option_6mo_excel():
    result = black_scholes(S=100, K=105, T=0.5, r=0.03, sigma=0.2, option_type="call")
    assert isinstance(result, float)
    assert abs(result - 4.1783) < 1e-3

def test_put_option_1yr_excel():
    result = black_scholes(S=50, K=45, T=1, r=0.02, sigma=0.25, option_type="put")
    assert isinstance(result, float)
    assert abs(result - 2.3078) < 1e-3

def test_call_option_atm_1yr():
    result = black_scholes(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type="call")
    assert isinstance(result, float)
    assert abs(result - 10.45) < 1e-2

def test_put_option_atm_1yr():
    result = black_scholes(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type="put")
    assert isinstance(result, float)
    assert abs(result - 5.573) < 1e-2

def test_invalid_negative_stock():
    result = black_scholes(S=-100, K=100, T=1, r=0.05, sigma=0.2, option_type="call")
    assert isinstance(result, str)
    assert result.startswith("Error:")

def test_invalid_option_type():
    result = black_scholes(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type="invalid")
    assert isinstance(result, str)
    assert result.startswith("Error:")

ipytest.run()

In [ ]:
# Gradio demo for black_scholes function
import gradio as gr

examples = [
    [100, 105, 0.5, 0.03, 0.2, "call"],  # Call option
    [50, 45, 1, 0.02, 0.25, "put"],       # Put option
    [100, 100, 1, 0.05, 0.2, "call"],     # ATM call
    [100, 100, 1, 0.05, 0.2, "put"],      # ATM put
]

demo = gr.Interface(
    fn=black_scholes,
    inputs=[
        gr.Number(label="S (Stock Price)", value=100),
        gr.Number(label="K (Strike Price)", value=105),
        gr.Number(label="T (Time to Expiration, years)", value=0.5),
        gr.Number(label="r (Risk-Free Rate)", value=0.03),
        gr.Number(label="sigma (Volatility)", value=0.2),
        gr.Dropdown(["call", "put"], label="Option Type", value="call"),
    ],
    outputs=gr.Number(label="Option Price"),
    examples=examples,
    description="Calculate the theoretical price of a European call or put option using the Black-Scholes formula.",
    flagging_mode="never",
)

demo.launch()